In [1]:
import os
import pickle

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import tqdm
from torch.nn import functional as fnn
from torch.utils import data
from torchvision import transforms

from hack_utils import ScaleMinSideToSize, CropCenter, TransformByKeys
from hack_utils import ThousandLandmarksDataset
from hack_utils import restore_landmarks_batch, create_submission
from hack_utils import stqdm

import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
DATA_DIR = '../data/'
LOCAL_DATA_DIR = '/work/local_data/made_cv/contest1'

In [3]:
GPUs = [0, 1, 2]

In [4]:
NUM_PTS = 971
CROP_SIZE = 224
BATCH_SIZE = 448 * len(GPUs)
NUM_WORKERS = 16

In [5]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [6]:
train_transforms = transforms.Compose([
    ScaleMinSideToSize((CROP_SIZE, CROP_SIZE)),
    CropCenter(CROP_SIZE),
    TransformByKeys(transforms.ToPILImage(), ("image",)),
    TransformByKeys(transforms.ToTensor(), ("image",)),
#     TransformByKeys(transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]), ("image",)),
])

In [7]:
train_dataset = ThousandLandmarksDataset(os.path.join(LOCAL_DATA_DIR, 'train'), train_transforms, split="train")
train_dataloader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, pin_memory=True,
                                   shuffle=True, drop_last=True)

Total lines (without header): 393930
Loading train dataset (315144 lines)


In [8]:
N_CHANNELS = 3

mean = torch.zeros(N_CHANNELS)
std = torch.zeros(N_CHANNELS)

for batch in stqdm(train_dataloader, total=len(train_dataloader)):
    images = batch["image"]
    for i in range(N_CHANNELS):
        mean[i] += images[:,i,:,:].mean()
        std[i] += images[:,i,:,:].std()

In [10]:
mean.div_(len(train_dataloader))
std.div_(len(train_dataloader))
print(mean, std)

tensor([0.4782, 0.3758, 0.3277]) tensor([0.3077, 0.2652, 0.2521])
